# Build model
<font color=blue>[包含tutorial的Build model和developer note的Modules]</font>
## 1. 什么是module，pytorch有哪些module类型？
 - module是构建神经网络的基础模块，pytorch用module来表达神经网络。
 - pytorch提供了一个modules库，也支持自定义modules。用他们可以很容易地构建多层神经网络。
   - 具体实现来看，<font color=green>**namespace**</font> **torch.nn**提供了layers, containers和utilities三种主要的module类型，并使用**tensor类型的nn.Parameter**作为modules parameter。
1. <font color=blue>**Layers：**</font>NN通过layers对数据进行操作。pytorch用modules来表达这些layers,比如conv, affine, pooling, normalization, transformer和loss functions等
2. <font color=blue>**containers：**</font>有3类container，nn.Module，nn.Sequential和holders of submodules。\
(1)**torch.nn.Module**。它是所有NN modules的base class，pytorch中所有的module都是**nn.Module**的子类\
(2)**torch.nn.Sequential**：以序列形式将1个或多个module顺序排列，体现了module的nestable\
(3)holders of submodules,其中：**nn.ModuleList，nn.ModuleDict**分别是以list和dictionary类型存储的module序列。**nn.ParamterList和nn.ParameterDict**分别是以list和dictionary形式存储的参数。
3. <font color=blue>**utilities：**</font>把一些数据处理的函数以modules的形式表达。

## 2. module的特点
- module和autograd system一起工作：modules使optimizer update参数非常方便。因为module能在autograd system的管理下自动完成requires_grad=True的tensor的梯度计算，optimizer可以在此基础上自动工作。
- pytorch中的module可以nest：每个神经网络模型自身都是一个module，该module又由其他modules(layers)构成。这种nest structure可以很方便的构造复杂的网络架构。
- **nn.Module**的子类会自动track参数，可以用两个method来查看：parameters()和named_parameters()
   - <font color=orange>module和autograd.Functional相比，可以自动维护module.parameters()中的参数。</font>
- 好用，而且也容易对module做各种调整：modules的save和restore都很直接，在CPU/GPU之间移动，做prune，quantize和其他很多操作都很方便

In [1]:
import os
import torch
import torch.nn as nn           # for torch.nn.Module
import torch.nn.functional as F # for the activation function
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

/home/roark/anaconda3/envs/231n/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/roark/anaconda3/envs/231n/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Using cuda device


## 3. 自定义module实现不同的功能
- 自定义model要定义为**nn.Module**的子类，每个子类必须定义<font color=blue>**\__init\__()和forward()**</font>两个method。
  - module通过他们package <font color=blue>state and computation</font> together。
    - 模型对input data的操作都放在forward() method中。forward()用来指定要执行的computation，用的operation是nn.autograd.Function的子类的实例。这些子类可以是pytorch定义好的，也可以是自定义的。
    - \__init__()中要对computation涉及的参数做register，登记后参数就会成为state的一部分，作为<font color=brown>learnable aspects of module's computation</font>。autograd system工作时就能track他们。
      - register的方法是在\__init\__()中将parameter定义为nn.Parameter的实例。
- module本身是callable的，call module会invoke forward function。
- Parameter class是torch.Tensor的子类，但他们可以被assigned as attributes of a Module。一旦实例化后，这些parameters就会被加到lists of the module's parameters。
  - 可以通过module.parameters()和model.namedparameters()来iterate throgh。

- <font color=green>**用nn.Mudule来实例化module时，只implement forward() method不用implement backward() method**。</font>因为：
   - <font color=blue>用nn.autograd.Function来实例化（自定义）Function时，要同时implement forward() and backward() methods</font> 
   - <font color=blue>autograd system会用Function中的backward来自动处理module中用到的function的backward pass。</font>

### 3.1 自定义一个简单的layer Module

In [2]:
class MyLinear(nn.Module):  # 必须是nn.Module的子类
    def __init__(self, in_features, out_features):
        super().__init__()

        # registering parameters: 参数定义成nn.Parameter的实例
        # 此时autograd会自动tracking并让optimizer在迭代时update
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(out_features))

  # implement forward() method
    def forward(self, input):
        return input @ self.weight + self.bias

In [3]:
model = MyLinear(4, 3)          
sample_input = torch.randn(4)  

# model is callable, calling invoke forward function
model(sample_input)

tensor([ 1.3376, -0.0370, -0.2683], grad_fn=<AddBackward0>)

In [4]:
## 遍历parameters()
for parameter in model.parameters():
    print(parameter)

print('\n')
    
## 遍历parameters named_parameters()
#  这里weights和bias是parameter的name
for parameter in model.named_parameters():
    print(parameter)

Parameter containing:
tensor([[-0.3550,  0.9415,  0.8648],
        [-0.2813, -0.8728, -0.7064],
        [-0.5670,  0.3550, -0.2009],
        [ 1.9295, -0.5165, -0.7180]], requires_grad=True)
Parameter containing:
tensor([-0.9009, -1.1516, -0.8664], requires_grad=True)


('weight', Parameter containing:
tensor([[-0.3550,  0.9415,  0.8648],
        [-0.2813, -0.8728, -0.7064],
        [-0.5670,  0.3550, -0.2009],
        [ 1.9295, -0.5165, -0.7180]], requires_grad=True))
('bias', Parameter containing:
tensor([-0.9009, -1.1516, -0.8664], requires_grad=True))


### 3.2 将modules作为network的基础模块(building blocks)
- modules contain other modules让他们成为实现复杂功能的building blocks。

#### 3.2.1 用nn.Sequential来chain together多个layer modules
- nn.Sequential会自动将上一层的输出传给下一层作为输入。<font color=red>但只在输入和输出都是单变量的情况时有效。</font>
- 除了下面例子中非常简单的案例，不建议直接用Sequential来定义module，因为sequential中用submodule的方式单一。<font color=green>最好自定义module，这样module中使用submodule来做computation的方式可以非常灵活。</font>

In [5]:
# nn.Sequential本身也是nn.Module的子类，所以实例化得到的也是module
net = nn.Sequential(
    MyLinear(4, 3),
    nn.ReLU(),
    MyLinear(3, 1)
)

simple_input = torch.randn(4)
net(sample_input)

tensor([0.2850], grad_fn=<AddBackward0>)

####  3.2.2 自定义module来搭建network：只用layer module
- 在__init__()中定义的submodule通常用torch.nn中或者自定义的layer module，对应神经网络中的layer。
  - **对比这里和前面用module类自定义layer module**：
    - 在用module自定义layer module时__init__()中定义的是forward computation中使用的parameters。此时，这些parameters被registered，从而被autograd system track到
    - 这里的__init__()中定义的是forward computation中使用的有parameters的 modules，这些modules中的parameters也会被registered。并且如果module不是简单的layer module，而是有submodule的话，submodule中的参数全都会被registered。
- 在__init__()中定义的submodule也可以用nn.ModuleList,nn.ModuleDict，他们可以实现动态定义submodule

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = MyLinear(4, 3)
        self.layer1 = MyLinear(3, 1)  # 定义了两个submodule
    
    def forward(self, x):
        x = self.layer0(x)
        x = F.relu(x)                 # relu不是submodule，是function
        x = self.layer1(x)
        return x

In [7]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = MyLinear(4, 3)
        self.relu = nn.ReLU()         # 用的nn中的ReLU module
        self.layer1 = MyLinear(3, 1)  # 定义了3个submodule
    
    def forward(self, x):
        x = self.layer0(x)
        x = self.relu(x)              # 这里relu是submodule
        x = self.layer1(x)
        return x
# 可以这么处理，但不必要，因为ReLU没有参数，没必要用module，就function就好

- **可以用children() or named_children()来iterate遍历module的immediate children**

In [8]:
net = Net()
for child in net.named_children():
    print(child)

('layer0', MyLinear())
('layer1', MyLinear())


In [9]:
# 对比前面例子中直接用tensor operation定义的module
# 此时module中没有child
for child in model.children():
    print(child)

In [10]:
# 也可以把relu处理成module
net2 = Net2()
for child in net2.named_children():
    print(child)

('layer0', MyLinear())
('relu', ReLU())
('layer1', MyLinear())


- 如果不止想查看immediate children，想深入看所有nested的module，就用**modules() and named_modules() recursively iterate through a module and its child modules**

In [11]:
class BigNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = MyLinear(5, 4)
        self.net = Net()
    def forward(self, x):
        return self.net(self.l1(x))

big_net = BigNet()

In [12]:
# 只看immediate children
for child in big_net.children():
    print(child)

MyLinear()
Net(
  (layer0): MyLinear()
  (layer1): MyLinear()
)


In [13]:
# 看所有nested children
for module in big_net.named_modules():
    print('-' * 52)
    print(module)

----------------------------------------------------
('', BigNet(
  (l1): MyLinear()
  (net): Net(
    (layer0): MyLinear()
    (layer1): MyLinear()
  )
))
----------------------------------------------------
('l1', MyLinear())
----------------------------------------------------
('net', Net(
  (layer0): MyLinear()
  (layer1): MyLinear()
))
----------------------------------------------------
('net.layer0', MyLinear())
----------------------------------------------------
('net.layer1', MyLinear())


####  3.2.3 自定义module来搭建network：用ModuleList/ModuleDict来dynamically define submodule
- 用ModuleList或者ModuleDict在自定义module的__init__()中定义submodule
  - ModuleList和ModuleDict都会自动register list/Dict中的submodule。实际上登记的是submodule中的参数。此时，calls to parameters() and named_parameters() will recursively include child parameters, 使得对network中所有parameters做optimization非常方便。

In [14]:
class DynamicNet(nn.Module):
    def __init__(self, num_layers):
        super().__init__()
        self.linears = nn.ModuleList(
            [MyLinear(4, 4) for _ in range(num_layers)])
        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU()
        })
        self.final = MyLinear(4, 1)
        
    def forward(self, x, act):
        for linear in self.linears:
            x = linear(x)
        x = self.activations[act](x)
        x = self.final(x)
        return x

dynamic_net = DynamicNet(3)
sample_input = torch.randn(4)
output = dynamic_net(sample_input, 'relu')

- **child module由__init__()中排列的module sequence决定，不由forward()实际执行的computation决定**

In [15]:
for module in dynamic_net.named_modules():
    print('-'*52)
    print(module)

----------------------------------------------------
('', DynamicNet(
  (linears): ModuleList(
    (0-2): 3 x MyLinear()
  )
  (activations): ModuleDict(
    (relu): ReLU()
    (lrelu): LeakyReLU(negative_slope=0.01)
  )
  (final): MyLinear()
))
----------------------------------------------------
('linears', ModuleList(
  (0-2): 3 x MyLinear()
))
----------------------------------------------------
('linears.0', MyLinear())
----------------------------------------------------
('linears.1', MyLinear())
----------------------------------------------------
('linears.2', MyLinear())
----------------------------------------------------
('activations', ModuleDict(
  (relu): ReLU()
  (lrelu): LeakyReLU(negative_slope=0.01)
))
----------------------------------------------------
('activations.relu', ReLU())
----------------------------------------------------
('activations.lrelu', LeakyReLU(negative_slope=0.01))
----------------------------------------------------
('final', MyLinear())


In [16]:
for parameter in dynamic_net.named_parameters():
    print('-'*68)
    print(parameter)

--------------------------------------------------------------------
('linears.0.weight', Parameter containing:
tensor([[-0.2599, -0.5645, -0.0587, -0.9620],
        [-0.4756,  0.3471,  0.2669, -0.8156],
        [ 0.7448, -0.2851, -0.7745,  0.4289],
        [-1.1384,  0.1852, -0.9535,  1.3573]], requires_grad=True))
--------------------------------------------------------------------
('linears.0.bias', Parameter containing:
tensor([-1.6436,  0.5007,  0.0061, -0.7417], requires_grad=True))
--------------------------------------------------------------------
('linears.1.weight', Parameter containing:
tensor([[-0.9521, -1.1884,  1.3178,  0.4964],
        [-0.4773, -0.6498,  1.3463,  0.4104],
        [-0.1620, -0.5115, -0.5420,  0.1848],
        [ 0.6219, -0.4513,  0.1677,  1.2817]], requires_grad=True))
--------------------------------------------------------------------
('linears.1.bias', Parameter containing:
tensor([ 0.7707,  0.3933, -1.1715,  0.5077], requires_grad=True))
------------

#### 3.2.4 移动参数的设备，改变参数精度，用.to()

In [17]:
# 将module移动到cuda
dynamic_net.to(device='cuda')

# 改变数据类型
dynamic_net.to(dtype=torch.float64)

dynamic_net(torch.randn(4, device='cuda', dtype=torch.float64), 'relu')

tensor([7.3285], device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)

#### 3.2.5 module和submodule可以apply任意函数，包括自定义函数
- module.apply(func)可以将任意函数recursively apply到module及其submodule上
- <font color=green>此时要注意判断func是否需要参与梯度计算，不需要的话要设置no_grad属性</font>

In [18]:
## 定义一个初始化整个module的Linear weights的函数
# no_grad()用来avoid tracking this computation in the autograd graph.
@torch.no_grad()
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight) # init操作用in-place节省空间
        m.bias.fill_(0.0)

# Apply the function recursively on the module and its submodules.
dynamic_net.apply(init_weights)

DynamicNet(
  (linears): ModuleList(
    (0-2): 3 x MyLinear()
  )
  (activations): ModuleDict(
    (relu): ReLU()
    (lrelu): LeakyReLU(negative_slope=0.01)
  )
  (final): MyLinear()
)

## 4. 使用module训练NN
**module有两种mode：trainning mode和evaluation mode**
1. module默认处于training mode。用training()和eval()可以改变module所处mode。
2. 如果module中有submodule在training mode和evaluation mode的时候输出不同，那么就应该在inference的时候将mode改为evaluation mode，比如batchnorm

In [19]:
# 新建network和optimizer
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-4, 
                            weight_decay=1e-2, momentum=0.9)

# trainging the netword
for _ in range(10000):
    input = torch.randn(4)
    output = net(input)
    loss = torch.abs(output) # 用abs做loss，会让weights趋于0
    
    net.zero_grad()
    loss.backward()
    optimizer.step()
    
# training完成后，将module转到eval mode
net.eval()

Net(
  (layer0): MyLinear()
  (layer1): MyLinear()
)

In [20]:
print(net.layer1.weight)

Parameter containing:
tensor([[-3.2114e-04],
        [ 1.4435e+00],
        [ 1.9327e+00]], requires_grad=True)


In [21]:
# 在training和evaluation mode下输出不同的例子
class ModalModule(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    if self.training:
      # Add a constant only in training mode.
      return x + 1.
    else:
      return x

m = ModalModule()
x = torch.randn(4)
print('training mode output: {}'.format(m(x)))

m.eval()
print('evaluation mode output: {}'.format(m(x)))

training mode output: tensor([ 1.7402, -0.7920,  0.6037,  1.0033])
evaluation mode output: tensor([ 0.7402, -1.7920, -0.3963,  0.0033])


## 5. module初始化
- **建议方式：先skip_init()method，之后自定义初始化方式。**
- **不建议使用**：
  - (1) 默认初始化，torch.nn提供的module中的parameter和浮点数buffer会在module实例化的时候初始化为存在CPU上的32位浮点数值。
  - (2) 在module实例化的时候设置对应的arguments

In [22]:
# 将module直接初始化到GPU上，参数类型为16位浮点数
m = nn.Linear(5, 3, device='cuda', dtype=torch.half)

In [23]:
# 除参数外，上述初始化方式也适用于floating-point buffers registered for the module
m = nn.BatchNorm2d(3, dtype=torch.half)
print(m.running_mean)

tensor([0., 0., 0.], dtype=torch.float16)


In [24]:
# 例：自定义参数初始化为正交矩阵
m = torch.nn.utils.skip_init(nn.Linear, 5, 3)
nn.init.orthogonal_(m.weight)

Parameter containing:
tensor([[-0.0164, -0.0865,  0.4707,  0.6579, -0.5813],
        [-0.0952, -0.2116,  0.5484,  0.2515,  0.7630],
        [ 0.8967,  0.4016,  0.1020,  0.1043,  0.1156]], requires_grad=True)

- <font color=blue>**使用skip_init() method要满足两个条件：**</font>
  1. module的constructor中必须有device kwarg，他要传给constructor中所有的parameters和buffers
  2. 除了nn.init中的初始化操作之外，parameters和buffers在constructor中不能参与任何其他computation。
  - 另外还有一个不是必须，单建议遵守的规则是：module的constructor中加上dtype  kwarg，传给constructor中定义的所有parameter和floating-point buffers 。

In [25]:
import torch
from torch import nn

class MyModule(torch.nn.Module):
  def __init__(self, foo, bar, device=None):
    super().__init__()

    # ==== Case 1: Module直接创建parameters. ====
    # 必须把device参数传给所有新建的parameters.
    self.param1 = nn.Parameter(torch.empty((foo, bar), device=device))
    self.register_parameter('param2', nn.Parameter(torch.empty(bar, device=device)))

    # 为了支持skip_init中用的meta device,除了torch.nn.init中定义的初始化操作外
    # 不能对module's constructor中直接定义的parameters做任何其他操作
    with torch.no_grad():
        nn.init.kaiming_uniform_(self.param1)
        nn.init.uniform_(self.param2)


    # ==== Case 2: Module中创建了submodules. ====
    # Pass device along recursively. 
    # All submodules will need to support them as well
    # 所有torch.nn提供的modules都满足条件
    self.fc = nn.Linear(bar, 5, device=device)

    # This also works with containers.
    self.linears = nn.Sequential(
        nn.Linear(5, 5, device=device),
        nn.Linear(5, 1, device=device)
    )


    # ==== Case 3: Module中创建了buffers. ====
    # Pass device along during buffer tensor creation.
    self.register_buffer('some_buffer', torch.ones(7, device=device))

## 6. torch自带module中的典型layers

### nn.Flatten
1. 参数：torch.nn.Flatten(start_dim=1, end_dim=-1)
2. 压缩[start_dim, end_dim]范围的dims
2. 默认将输入的data压成2维数据，保留原第一维，压缩剩下的维度，比如输出(N, D)

In [26]:
input_image = torch.rand(3,28,28)
print(input_image.size())

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

flatten2 = nn.Flatten(0, 1)  # 压缩[0, 1]范围的dims
flat_image2 = flatten2(input_image)
print(flat_image2.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([84, 28])


### nn.Linear
1. affine layer
2. 参数：torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
   · in_features (int) – size of each input sample
   · out_features (int) – size of each output sample
   · bias (bool)取False时, 就不会learn bias. Default: True

In [27]:
layer1 = nn.Linear(in_features=28*28, out_features=6)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 6])


### nn.ReLU

In [28]:
print(f"Before ReLU:\n {hidden1}\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU:\n {hidden1}")

Before ReLU:
 tensor([[ 0.3486, -0.2854,  0.5229, -0.0191,  0.2855, -0.2265],
        [ 0.5093,  0.0711,  0.3753,  0.0769,  0.0072, -0.5286],
        [ 0.5357, -0.4205,  0.2364, -0.2208, -0.0226, -0.5407]],
       grad_fn=<AddmmBackward0>)

After ReLU:
 tensor([[0.3486, 0.0000, 0.5229, 0.0000, 0.2855, 0.0000],
        [0.5093, 0.0711, 0.3753, 0.0769, 0.0072, 0.0000],
        [0.5357, 0.0000, 0.2364, 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)


### nn.Sequential
1. an ordered container of modules.
2. 数据会按照Sequential中定义的layer顺序做处理

In [29]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(6, 10)
)
input_image = torch.rand(3,28,28)
scores = seq_modules(input_image)

softmax = nn.Softmax(dim=1)
pred_probab = softmax(scores)